# File Retrieval

This script downloads required meteorological files from copernicus using the following parameters:
 - Center: Met Office
 - System: 12
 - Variables: 2m Air Temperature, Mean Sea Level Pressure, Total Precipiation
 - Leadtime: 720-1440 hours (24 hour increments)

In [2]:
from tqdm.notebook import tqdm
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
import xarray as xr
import cdsapi
import os

First, , where should we be sending the files? and define the meteorological variables.

In [3]:
output_folder = "E:/31-12-2020/forecast-data/"
meteo_vars = ['2m_temperature', 'mean_sea_level_pressure', 'total_precipitation']

In [20]:
def get_max_min_leadtimes(fmonth, fyear, leadtime):
    """ Determines how many hours are there before a day to be forecasted and how many hours should we be forecasting for.
        Parameters:
            fmonth - Month being forecasted from.
            fyear - Year being forecasted from.
            leadtime - Number of months leadtime to give.
        Returns: Number of hours to first day intended for forecasting and number hours to last day of forecasting (inclusive)."""
    fdate = date(fyear, fmonth, 1)
    ddate = fdate + relativedelta(months=leadtime)
    sdate = ddate + relativedelta(months=1) - relativedelta(days=1)
    min_lt = ((ddate - fdate).days * 24)
    max_lt = ((sdate - fdate).days * 24)
    return min_lt, max_lt

def download_files(meteo_vars, output_folder, leadtime=1):
    """ Downloads the meteorological data in monthly files with the given leadtime. This stores them
        with a file name based on the date of the forecast."""
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    for y in tqdm(range(1993, 2016)):
        for m in range(1, 13):
            if (y == 1993) and (m == 1):
                continue
            file_name = output_folder + "{}-{}.grib".format(m, y)
            # Get the leadtimes
            mi, ma = get_max_min_leadtimes(m, y, leadtime)
            leadtimes = list(range(mi, ma+1, 24))
            if not os.path.exists(file_name):
                print("Downloading {}-{}".format(m, y))
                if y == 1993:
                    download_year(meteo_vars, m, y, leadtimes, file_name)
                else:
                    download_year(meteo_vars, m, y, leadtimes, file_name)
            else:
                print("Already exists: {}-{}".format(m, y))
        
def download_year(meteo_vars, month, year, leadtimes, output_file, system='12'):
    """ Downloads a forecast based on the provided parameters, saving it in the intended file. """
    c = cdsapi.Client()
    c.retrieve(
        'seasonal-original-single-levels',
        {
            'originating_centre': 'ukmo',
            'system': system,
            'variable': meteo_vars,
            'year': year,
            'month': month,
            'day': '01',
            'leadtime_hour': leadtimes,
            'area': [
                70, -100, 10,
                20,
            ],
            'format': 'grib',
        },
        output_file)
    
download_files(meteo_vars, output_folder)

2021-01-17 12:45:30,900 INFO Welcome to the CDS


Already exists: 2-1993
Already exists: 3-1993
Already exists: 4-1993
Already exists: 5-1993
Already exists: 6-1993
Already exists: 7-1993
Already exists: 8-1993
Already exists: 9-1993
Already exists: 10-1993
Already exists: 11-1993
Already exists: 12-1993
Already exists: 1-1994
Already exists: 2-1994
Already exists: 3-1994
Already exists: 4-1994
Already exists: 5-1994
Already exists: 6-1994
Already exists: 7-1994
Already exists: 8-1994
Already exists: 9-1994
Already exists: 10-1994
Already exists: 11-1994
Already exists: 12-1994
Already exists: 1-1995
Already exists: 2-1995
Already exists: 3-1995
Already exists: 4-1995
Already exists: 5-1995
Already exists: 6-1995
Already exists: 7-1995
Already exists: 8-1995
Already exists: 9-1995


2021-01-17 12:45:30,902 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-original-single-levels
2021-01-17 12:45:31,185 INFO Request is queued
2021-01-17 12:45:32,209 INFO Request is running
2021-01-17 12:45:39,434 INFO Request is completed
2021-01-17 12:45:39,434 INFO Downloading https://download-0007.copernicus-climate.eu/cache-compute-0007/cache/data0/adaptor.mars.external-1610887532.0635903-28374-6-6d9b365c-72fd-4548-bba6-bd2da7b2fa9a.grib to E:/31-12-2020/forecast-data/10-1995.grib (8.9M)
2021-01-17 12:45:43,504 INFO Download rate 2.2M/s                                                                      


Already exists: 11-1995
Already exists: 12-1995
Already exists: 1-1996
Already exists: 2-1996
Already exists: 3-1996
Already exists: 4-1996
Already exists: 5-1996
Already exists: 6-1996
Already exists: 7-1996
Already exists: 8-1996
Already exists: 9-1996
Already exists: 10-1996
Already exists: 11-1996
Already exists: 12-1996
Already exists: 1-1997
Already exists: 2-1997
Already exists: 3-1997
Already exists: 4-1997
Already exists: 5-1997
Already exists: 6-1997
Already exists: 7-1997
Already exists: 8-1997
Already exists: 9-1997
Already exists: 10-1997
Already exists: 11-1997
Already exists: 12-1997
Already exists: 1-1998
Already exists: 2-1998
Already exists: 3-1998
Already exists: 4-1998
Already exists: 5-1998
Already exists: 6-1998
Already exists: 7-1998
Already exists: 8-1998
Already exists: 9-1998
Already exists: 10-1998
Already exists: 11-1998
Already exists: 12-1998
Already exists: 1-1999
Already exists: 2-1999
Already exists: 3-1999
Already exists: 4-1999
Already exists: 5-1999


## Extract Relevant Data

Next, from the forecasted meteorological data we extract the information into files describing the month being forecasted. E.g 01-2020 with a leadtime of 720-1440 will be stored in 02-2020.

In [24]:
def extract_data(meteo_folder, output_folder, mvars=['msl', 't2m', 'tp']):
    """ Extracts the meteorological data from the GRIB files in 'meteo_folder' and stores them in independent files
        named after the dates for the data they represent. E.G file 02-2020 at a leadtime of 1 month represents 03-2020."""
    _, _, files = next(os.walk(meteo_folder))
    data_safety = []
    for f in tqdm(files):
        if f[-5:] == ".grib":
            input_fname = meteo_folder + f
            try:
                ds = xr.open_dataset(input_fname, engine='cfgrib')
                for v in mvars:
                    output_fname = output_folder + v + "/" + f[:-5] +".npy"
                    #if not os.path.exists(output_fname):
                    if len(ds[v].shape) > 3:
                        data = np.mean(ds[v], axis=0)
                        if len(data.shape) == 4:
                            print(output_fname)
                            data = data[0, :, :, :]
                    else:
                        data = np.array(ds[v])
                    data_safety.append([np.max(data), np.min(data), data.shape, f])
                    np.save(output_fname, data)
            except Exception as e:
                print("Failed to open file: {}".format(input_fname))
                print(e)
            finally:
                ds.close()
    return data_safety
safety = extract_data("E:/31-12-2020/forecast-data/", "E:/31-12-2020/forecastee-data/")

2021-01-17 12:56:11,168 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:11,170 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:11,183 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:11,184 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:11,200 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:11,201 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:12,148 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:12,149 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:12,160 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:12,161 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:12,174 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:12,175 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:13,090 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:13,092 INFO missing from GRIB stre

2021-01-17 12:56:33,075 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:33,986 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:33,986 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:34,005 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:34,005 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:34,016 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:34,016 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:34,983 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:34,984 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:34,997 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:34,998 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:35,009 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:35,010 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:35,845 INFO missing from GRIB stre

2021-01-17 12:56:52,373 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:52,373 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:53,246 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:53,246 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:53,267 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:53,267 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:53,274 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:53,274 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:54,152 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:54,152 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:54,158 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:54,158 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:56:54,173 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:56:54,173 INFO missing from GRIB stre

2021-01-17 12:57:14,034 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:57:14,049 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:57:14,049 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:57:15,074 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:57:15,074 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:57:15,089 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:57:15,090 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:57:15,104 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:57:15,105 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:57:16,271 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:57:16,272 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:57:16,285 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:57:16,286 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:57:16,298 INFO missing from GRIB stre

2021-01-17 12:57:37,507 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:57:37,509 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:57:37,523 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:57:37,525 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:57:39,563 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:57:39,564 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:57:39,633 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:57:39,635 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:57:39,652 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:57:39,653 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:57:41,522 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:57:41,524 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 12:57:41,584 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 12:57:41,585 INFO missing from GRIB stre

...

2021-01-17 13:00:12,578 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:12,579 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:12,642 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:12,644 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:12,659 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:12,660 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:13,830 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:13,832 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:13,869 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:13,871 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:13,884 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:13,886 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:14,978 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:14,980 INFO missing from GRIB stre

2021-01-17 13:00:34,492 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:35,602 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:35,603 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:35,669 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:35,671 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:35,683 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:35,684 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:36,820 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:36,823 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:36,876 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:36,878 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:36,890 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:36,891 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:38,086 INFO missing from GRIB stre

2021-01-17 13:00:57,574 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:57,575 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:58,653 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:58,654 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:58,691 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:58,692 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:58,705 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:58,706 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:59,890 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:59,891 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:59,965 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:59,967 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:00:59,979 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:00:59,981 INFO missing from GRIB stre

2021-01-17 13:01:20,572 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:01:20,585 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:01:20,587 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:01:21,721 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:01:21,722 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:01:21,782 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:01:21,785 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:01:21,800 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:01:21,802 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:01:22,880 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:01:22,882 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:01:22,928 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:01:22,930 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:01:22,942 INFO missing from GRIB stre

2021-01-17 13:01:44,125 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:01:44,126 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:01:44,138 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:01:44,141 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:01:45,254 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:01:45,256 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:01:45,317 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:01:45,318 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:01:45,330 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:01:45,331 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:01:46,367 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:01:46,369 INFO missing from GRIB stream: 'frequencyNumber'
2021-01-17 13:01:46,413 INFO missing from GRIB stream: 'directionNumber'
2021-01-17 13:01:46,415 INFO missing from GRIB stre

In [25]:
def rename_data(data_folder, leadtime=1):
    """ Renames the files to represent the dates for which they actually match. """
    _, _, files = next(os.walk(data_folder))
    for f in tqdm(files):
        date_of_forecast = date(int(f.split('-')[1].split('.')[0]), int(f.split('-')[0]), 1)
        date_forecasted = date_of_forecast + relativedelta(months=1)
        output_file = data_folder + "forecasted-months/{}-{}.npy".format(date_forecasted.month, date_forecasted.year)
        if not os.path.exists(output_file):
            os.rename(data_folder + f, output_file)
rename_data("E:/31-12-2020/forecastee-data/msl/")
rename_data("E:/31-12-2020/forecastee-data/t2m/")
rename_data("E:/31-12-2020/forecastee-data/tp/")